In [1]:
from google import genai
from google.genai import types
from db_utils import csv_read,query_database
import base64
import re
import os

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'D:\api\gemini2.0.json'

In [3]:
client = genai.Client(
        vertexai=True,
        project="gen-lang-client-0000496465",
        location="us-central1"
    )

In [4]:
def extract_sql_queries(sql_response):
    matches = re.findall(r'SELECT.*?;', sql_response, re.IGNORECASE | re.DOTALL) 
    if matches:
        # 移除每個匹配項中的換行符號 
        matches = [match.replace('\n', ' ') for match in matches]
        return matches 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [5]:
def generate_sql_from_nl(user_input):
    csv_string = csv_read()
    system_prompt = f'''
        1. 你是一個財務資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文，結束請記得給分號。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        4. MYSQL code中請不要出現資料庫中沒有的英文名稱。
        5. 請務必生成完整SQL語法，並一定要在首欄加入年或季度的欄位，例如seasonid。
       '''
    user_prompt = f'''
        1.資料庫結構、表名稱、欄位名稱請務必參考以下內容:{csv_string}
        2.一定不能生成沒有的sql表名與欄名，欄位名稱請盡量比對中文對照!
        3.請務必要判斷是否需要跨表查詢，如需要請使用join指令。
        4.另外時間只有年與季度seasonsID，例如2010年第一季格式為201001、2010年第二季格式為201002、
        2020年第三季格式為202003、2020年第四季格式為202004
                   '''
    model= "gemini-2.0-flash-exp"
    
    generation_config = types.GenerateContentConfig(
            max_output_tokens=150,
            temperature=0.1,
            response_modalities = ["TEXT"]
            )
    
    contents = [types.Content(parts=[
            types.Part.from_text(system_prompt + user_input + user_prompt)
        ], role="user")
    ]

    response = client.models.generate_content(
            model = model,
            contents = contents,
            config = generation_config)

    sql_response = response.text
    print(f"回答:{sql_response}")
    return sql_response


In [6]:
def classify_user_input(user_input:str):
    # 初次調用判斷是否需要function calling

    model= "gemini-2.0-flash-exp"
    
    prompt = '''1.請判斷使用者的輸入是否需要連線到「財務報表」資料庫查詢，若需要請回答finvision即可!
                2.若使用者的輸入是關於分析財務指標，請返回analysis即可!
                3.若都不屬於查詢與分析範圍，只要回答NO即可!
                4.財務報表資料庫內容包含資產負債表、損益表、現金流量表，其餘公司資料都沒有。'''

    content = [types.Content(parts=[
            types.Part.from_text(prompt)
            ], role="user")
            ]
    
    
    response = client.models.generate_content(
            model = model,
            contents = content)
    
    
    print('===== 初次調用GPT判斷 =====')
    if 'finvision' in response.text:
        sql_response = generate_sql_from_nl(user_input) 
        sql_query = extract_sql_queries(sql_response)
        print(f'生成的SQL指令: {sql_query}') 
        # 執行GPT生成的SQL查詢 
        report = query_database(sql_query)
        print(f'查詢結果: {report}')
        return(f"finvision select :{report}")

    else: 
        # 一般對話處理
        return(response.text)

In [ ]:
def main():
    user_input ="請給我2023年各季度之利息收入、法定盈餘公積、銷貨收入"
    report = classify_user_input(user_input)
    return(report)

In [7]:
main()

===== 初次調用GPT判斷 =====
回答:```sql
SELECT
    s.seasonsID,
    pl.II AS interest_income,
    bs.LR AS legal_surplus_reserve,
    pl.NSR AS sales_revenue
FROM
    seasons s
LEFT JOIN
    bs ON s.seasonsID = bs.seasonsID
LEFT JOIN
    pl ON s.seasonsID = pl.seasonsID
WHERE
    SUBSTR(s.seasonsID, 1, 4) = '2023';
```

生成的SQL指令: ["SELECT     s.seasonsID,     pl.II AS interest_income,     bs.LR AS legal_surplus_reserve,     pl.NSR AS sales_revenue FROM     seasons s LEFT JOIN     bs ON s.seasonsID = bs.seasonsID LEFT JOIN     pl ON s.seasonsID = pl.seasonsID WHERE     SUBSTR(s.seasonsID, 1, 4) = '2023';"]
連線成功
查詢結果: [[{"seasonsID": "202301", "interest_income": 13617818, "legal_surplus_reserve": 311146899, "sales_revenue": 0}, {"seasonsID": "202302", "interest_income": 27289839, "legal_surplus_reserve": 311146899, "sales_revenue": 0}, {"seasonsID": "202303", "interest_income": 42885113, "legal_surplus_reserve": 311146899, "sales_revenue": 0}, {"seasonsID": "202304", "interest_income": 60293901,

'finvision select'

以下是一般對話連線測試

In [13]:
def generate():
    client = genai.Client(
        vertexai=True,
        project="gen-lang-client-0000496465",
        location="us-central1"
    )

    model = "gemini-2.0-flash-exp"

    contents = [types.Content(parts=[
            types.Part.from_text("請問python是甚麼?")
        ], role="user")
    ]
    generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"])

    response = client.models.generate_content(
            model = model,
            contents = contents,
            config = generate_content_config)
    return(response.text)

In [14]:
response = generate()
print(f'回覆:{response}')

回覆:Python 是一種**高階、通用型的程式語言**，它以其**易讀性、簡潔的語法**和**強大的功能**而聞名。以下是 Python 的一些關鍵特點：

**主要特點：**

* **易於學習和閱讀：** Python 的語法非常接近英文，這使得它對初學者來說非常容易上手。它強調程式碼的可讀性，使用縮排來組織程式碼結構，而不是像其他語言那樣使用大括號。
* **多用途：** Python 可以用於各種不同的應用領域，包括：
    * **網頁開發：** 使用框架如 Django 和 Flask 來建立網站和 Web 應用程式。
    * **資料科學與機器學習：** 擁有豐富的函式庫如 NumPy、Pandas、Scikit-learn 和 TensorFlow，使其成為資料分析、機器學習和人工智慧的首選語言。
    * **自動化腳本：** 可以編寫腳本來自動執行重複性的任務，如檔案處理、網路管理等。
    * **遊戲開發：** 可以使用 Pygame 等函式庫來開發簡單的遊戲。
    * **科學計算與工程：** 可以使用 SciPy 等函式庫來進行數值計算、模擬和資料視覺化。
    * **桌面應用程式：** 可以使用 Tkinter、PyQt 等函式庫來開發圖形使用者介面應用程式。
* **直譯式語言：** Python 是一種直譯式語言，這意味著程式碼在執行時會逐行被翻譯成機器碼，不需要事先編譯。這使得開發過程更加快速和靈活。
* **動態類型：** Python 是一種動態類型語言，這表示你不需要明確聲明變數的類型，變數的類型會在執行時自動推斷。
* **大型標準函式庫：** Python 擁有一個非常豐富的標準函式庫，提供了許多內建的函數和模組，可以方便地完成各種任務，例如檔案處理、網路連線、資料庫操作等。
* **開源且跨平台：** Python 是開源的，可以免費使用和修改。它也可以在不同的作業系統上運行，例如 Windows、macOS 和 Linux。
* **龐大的社群：** Python 擁有一個龐大的活躍社群，提供了大量的教程、文件和支援，這使得學習和使用 Python 更加容易。

**總結：**

Python 是一種功能強大且用途廣泛的程式語言，它以其易讀性、靈活性和龐大的社群而聞名。無論你是初學者還是有經驗的開